In [1]:
import pandas as pd
import re
#set ups
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
# importing package 
import matplotlib.pyplot as plt 
import numpy as np 
import requests

#setups
%matplotlib inline


In [2]:
df=pd.read_csv("listings_full.csv")

## Variables used !!!!!

listing : df containing only the listings about entire rooms <br>
true_codes : a list that onlu has the codes of the <br>
airbnb :<br>
hosts_multiple :<br>
per_area_per_night :<br>

## Doing all the checks needed

No much cleaning is needed so we are going to do initial chscks before we strat the analysis

In [3]:
df.shape

(13182, 75)

In [4]:
df.dtypes

id                                                int64
listing_url                                      object
scrape_id                                         int64
last_scraped                                     object
source                                           object
name                                             object
description                                     float64
neighborhood_overview                            object
picture_url                                      object
host_id                                           int64
host_url                                         object
host_name                                        object
host_since                                       object
host_location                                    object
host_about                                       object
host_response_time                               object
host_response_rate                               object
host_acceptance_rate                            

In [5]:
# keeping the columns i am interested in
df=df[['id',
       'name', 
       'host_id',
       'host_name', 
       'host_since',
       'host_location', 
       'host_is_superhost',
       'host_neighbourhood', 
       'host_listings_count',
       'host_total_listings_count',
       'neighbourhood',
       'neighbourhood_cleansed',
       'latitude', 'longitude',
       'property_type',
       'room_type', 
       'price',
       'minimum_nights', 
       'maximum_nights',
       'minimum_minimum_nights',
       'maximum_minimum_nights',
       'minimum_maximum_nights',
       'maximum_maximum_nights',
       'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 
       'availability_365', 
       'review_scores_rating',
       'review_scores_accuracy',
       'license',
       'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms']].copy()

In [6]:
df.describe()

,id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_365,review_scores_rating,review_scores_accuracy,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
count,1.318200e+04,1.318200e+04,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,11075.000000,11075.000000,13182.000000,13182.000000,13182.000000,13182.000000
mean,3.954590e+17,2.166796e+08,20.644895,32.083447,37.980172,23.731728,3.151267,571.466925,3.197390,4.595812,757.426567,781.464421,4.157988,767.141405,222.418146,4.737552,4.783382,12.034441,10.849416,1.031027,0.071082
std,4.270628e+17,1.732957e+08,61.423587,136.062916,0.013427,0.012480,16.231489,457.941252,18.416376,26.587753,449.324853,437.633288,24.276247,440.906213,136.667965,0.431216,0.394101,19.431829,18.978576,4.439618,0.850407
min,1.059500e+04,1.805000e+03,1.000000,1.000000,37.950322,23.697700,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,3.166800e+07,6.022029e+07,1.000000,2.000000,37.969140,23.723970,1.000000,90.000000,1.000000,1.000000,365.000000,365.000000,1.000000,365.000000,88.000000,4.680000,4.750000,1.000000,1.000000,0.000000,0.000000
50%,5.312285e+07,1.821563e+08,5.000000,6.000000,37.978573,23.729719,2.000000,365.000000,2.000000,2.000000,1125.000000,1125.000000,2.000000,1125.000000,268.000000,4.860000,4.890000,4.000000,3.000000,0.000000,0.000000
75%,8.516889e+17,3.696750e+08,20.000000,23.000000,37.988298,23.737657,2.000000,1125.000000,2.000000,3.000000,1125.000000,1125.000000,2.300000,1125.000000,348.000000,4.980000,5.000000,14.000000,12.000000,0.000000,0.000000
max,1.052831e+18,5.516695e+08,2444.000000,5529.000000,38.030830,23.780220,800.000000,4444.000000,800.000000,1000.000000,4444.000000,4444.000000,800.000000,4444.000000,365.000000,5.000000,5.000000,121.000000,121.000000,44.000000,15.000000


# Questions

### How many hosts have more than two listings?

We need only the cases where a host povides an entire home. Hotel rooms are not included in this part of the analysis as we consider them to not take part in the multi-owning aspect. (They will be used in the map in order to see how much of Athens is taken up by enterprise-styled places.) 

"Rooms are great for guests who prefer a little privacy, yet still want to meet someone new and experience the area like a local. With a room, guests will have their own bedroom in a home, plus access to spaces that may be shared with others. For example, guests might walk through indoor spaces that another person may be sharing in order to get to their room.

What is required to be displayed as a Room to guests?
To be eligible to be displayed to guests as a room on Airbnb – for example, in search, in the Rooms category, on your listing page and otherwise – a listing should meet the following requirements:

The space listed is a bedroom, with a door, that is intended to be exclusively used by the guest during their stay
The guest shares access to at least one or more of the following common spaces: bathroom, kitchen, or living room"


In [7]:
# keeping only the entire rooms
ent_room=df[df.room_type.str.contains('Entire',case=False)].copy().reset_index()
listing = pd.DataFrame(ent_room.host_id.value_counts(ascending=False)).copy().reset_index()

In [8]:
# categorizing how many properties each host has 
# based on the greek legislation, from 3 and up the categorization changes
for i,item in listing.iterrows():
    if item['count'] >= 3:
        listing.at[i, 'many_properties'] = 'true' 
        
    else:
        listing.at[i, 'many_properties'] = 'false' 

In [9]:
# what precentage of the hosts have than 2 properties?
listing['many_properties'].value_counts(normalize=True)*100

many_properties
false    85.806567
true     14.193433
Name: proportion, dtype: float64

Let's compare this precentage with what Airbnb says it is the total listings of the host. Airbnb does not make a differantiation of the places so it can be rooms, entire places etc. 

In [10]:
airbnb = df[['host_id', 'host_listings_count']].copy()

In [11]:
for i,item in airbnb.iterrows():
    if item['host_listings_count'] >= 3:
       
        airbnb.at[i, 'many_properties'] = 'true'
    else:
        airbnb.at[i, 'many_properties'] = 'false' 

In [12]:
airbnb['many_properties'].value_counts(normalize=True)*100

many_properties
true     63.321196
false    36.678804
Name: proportion, dtype: float64

The percentage is  four times higher but it is not very safe to use for the analysis. We can keep it in mind. <br>
What does it mean though? That there is a great possibility these people or enterprises own places in other cities that are not visible in the present dataset.

### Let's check in what areas we can see most of these listings.

In [56]:
len(listing)

5573

In [13]:
# keeping only the values that are true and using it to identify these hosts in the general dataset
type(listing[listing.many_properties.str.contains('true')].host_id)

pandas.core.series.Series

In [14]:
# this is a series, we need to turn it in a list
true_codes=listing[listing.many_properties.str.contains('true')].host_id.to_list()

In [15]:
len(true_codes)

791

In [16]:
# making a dataset that keeps the rows with the true values and one with the false
result_multiple = []
result_single = []

for i, item in df.iterrows():
    if item['host_id'] in true_codes:
        multi = df.iloc[i]
        result_multiple.append(multi)
        
    else:
        non_multi = df.iloc[i]
        result_single.append(non_multi)
        
hosts_multiple=pd.DataFrame(result_multiple).copy().reset_index().drop(['index'],axis=1)
hosts_single=pd.DataFrame(result_single).copy().reset_index().drop(['index'],axis=1)

In [17]:
len(hosts_multiple)

6691

In [20]:
# in which neighbourhoods do we see more airbnb businesses
hm=hosts_multiple['neighbourhood_cleansed'].value_counts()
hm=pd.DataFrame(hm).reset_index()

In [18]:
len(hosts_single)

6491

In [19]:
# same for all the "under 2"
hs=hosts_single['neighbourhood_cleansed'].value_counts()
hs=pd.DataFrame(hs).reset_index()

In [46]:
# hs.merge(hm,on="neighbourhood_cleansed",how="outer").copy()

#### Question: <br>Does this mean that more houses in central Athens are owned by business like hosts than owned by non-business hosts?
Even though we see a smaller number of hosts having businesses, they do own more houses than the non businesses do. Of course that makes sense, because the do own more houses by definition. Does that mean a few people own most of the houses in Athens... Is that a strech of a conclusion?

### These are the areas with the precentage of listings

In [45]:
hosts_multiple['neighbourhood_cleansed'].value_counts(normalize=True)*100

neighbourhood_cleansed
ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ              23.748319
ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ                  9.804215
ΝΕΟΣ ΚΟΣΜΟΣ                          7.487670
ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ             7.263488
ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ     6.321925
ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ                      4.289344
ΠΑΓΚΡΑΤΙ                             3.826035
ΑΜΠΕΛΟΚΗΠΟΙ                          3.018981
ΑΚΡΟΠΟΛΗ                             2.944253
ΚΕΡΑΜΕΙΚΟΣ                           2.929308
ΚΟΛΩΝΑΚΙ                             2.914363
ΘΗΣΕΙΟ                               2.899417
ΠΕΤΡΑΛΩΝΑ                            2.660290
ΖΑΠΠΕΙΟ                              1.987745
ΣΤΑΔΙΟ                               1.763563
ΚΥΨΕΛΗ                               1.673890
ΓΟΥΔΙ                                1.643999
ΙΛΙΣΙΑ                               1.524436
ΝΙΡΒΑΝΑ                              1.255418
ΛΥΚΑΒΗΤΤΟΣ                           1.016291
ΠΑΤΗΣΙΑ                              0.986400
ΓΚΑΖΙ      

### Let's see the average price per night per area

In [48]:
# prices must be cleaned and integered
hosts_multiple['price_clean'] = hosts_multiple['price'].str.replace('$', '').str.replace(',', '').astype(float)

In [49]:
hosts_multiple.price_clean.mean()

119.98799574532745

In [50]:
hosts_multiple.price_clean.median()

72.0

In [51]:
# what is the average price per night for each area (multiple listings)?
per_area_per_night=hosts_multiple.groupby('neighbourhood_cleansed', as_index=False)['price_clean'].mean().round().sort_values(by='price_clean',ascending=False)
per_area_per_night

,neighbourhood_cleansed,price_clean
5,ΑΚΡΟΠΟΛΗ,271.0
26,ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ,254.0
16,ΖΑΠΠΕΙΟ,244.0
36,ΠΟΛΥΓΩΝΟ,134.0
21,ΚΟΛΩΝΑΚΙ,132.0
15,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,128.0
0,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,123.0
23,ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ,121.0
17,ΘΗΣΕΙΟ,116.0
19,ΚΕΡΑΜΕΙΚΟΣ,115.0


### Same process for the ones with less than 3 listings to check the difference in pricing

In [59]:
hosts_single['price_clean'] = hosts_single['price'].str.replace('$', '').str.replace(',', '').astype(float)

In [60]:
hosts_single.price_clean.mean()

90.17658380112269

In [61]:
hosts_single.price_clean.median()

60.0

In [62]:
# what is the average price per night for each area (single listing)?
per_area_per_night_single=hosts_single.groupby('neighbourhood_cleansed', as_index=False)['price_clean'].mean().round().sort_values(by='price_clean',ascending=False)
per_area_per_night_single

,neighbourhood_cleansed,price_clean
39,ΡΗΓΙΛΛΗΣ,654.0
9,ΒΟΤΑΝΙΚΟΣ,169.0
42,ΣΤΑΔΙΟ,141.0
15,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,139.0
19,ΚΕΡΑΜΕΙΚΟΣ,118.0
21,ΚΟΛΩΝΑΚΙ,117.0
5,ΑΚΡΟΠΟΛΗ,109.0
25,ΛΥΚΑΒΗΤΤΟΣ,109.0
23,ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ,101.0
17,ΘΗΣΕΙΟ,101.0


### Not sure if that is correct

In [52]:
# i want to find which prices are greater than the total median of all areas
for i,item in per_area_per_night.iterrows():
    if item['price_clean']>=72:
           per_area_per_night.at[i, 'greater_than_the_average'] = 'yes'
    else:
        per_area_per_night.at[i, 'greater_than_the_average'] = 'no'

In [53]:
per_area_per_night["greater_than_the_average"].value_counts(normalize=True)*100

greater_than_the_average
yes    59.52381
no     40.47619
Name: proportion, dtype: float64

AND MAKE THE MAP ON MONDAY AAAAAAA

#### Should i also find the price per night for longterm stay?

In [ ]:
# what precentage of these are greater than the average
# what is the "per night" in rent prices